In [ ]:
!ls

orchid_test.csv  orchid_train.csv  orchid_val.csv  sample_data


In [ ]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libxtst6
Suggested packages:
  libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 30.8 MB of archives.
After this operation, 104 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxtst6 amd64 2:1.2.3-1build4 [13.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 openjdk-8-jre-headless amd64 8u402-ga-2ubuntu1~22.04 [30.8 MB]
Fetched 30.8 MB in 2min 13s (232 kB/s)
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.

In [ ]:
train_raw_data=  sc.textFile("orchid_train.csv")
train_str_data= train_raw_data.map(lambda x:x.split(','))
train_data = train_str_data.map(lambda x:[float(y)for y in x])
print(train_data.take(20)[0][-5:])
print(train_data.take(20)[9][-5:])
print(train_data.take(20)[18][-5:])

[0.1933087557554245, -0.0045260111801326275, 0.1272827833890915, -0.2245592623949051, 0.0]
[0.12061325460672379, -0.23026636242866516, -0.023545075207948685, -0.13828127086162567, 1.0]
[0.36595019698143005, -0.5046675801277161, -0.5441071391105652, -0.08429872989654541, 2.0]


In [ ]:
n_cases =train_data.count()
label_distinct =train_data.map(lambda x:x[-1]).distinct()
n_classes_ =label_distinct.count()
print('num of cases= ',n_cases)
print('num of classes= ',n_classes_)

num of cases=  1533
num of classes=  219


In [ ]:
val_raw_data=  sc.textFile("orchid_val.csv")
val_str_data= val_raw_data.map(lambda x:x.split(','))
val_data = val_str_data.map(lambda x:[float(y)for y in x])

test_raw_data=  sc.textFile("orchid_test.csv")
test_str_data= test_raw_data.map(lambda x:x.split(','))
test_data = test_str_data.map(lambda x:[float(y)for y in x])

In [ ]:
from pyspark.mllib.regression import LabeledPoint
lp_train =train_data.map(lambda x:LabeledPoint(x[-1],x[:-1]) )
lp_val = val_data.map(lambda x:LabeledPoint(x[-1],x[:-1]) )
lp_test = test_data.map(lambda x:LabeledPoint(x[-1],x[:-1]) )

In [ ]:
for i in range(5):
  data_col =lp_train.map(lambda lp:lp.features[i])
  print("第",i+1,"個特徵",data_col.stats())


第 1 個特徵 (count: 1533, mean: -0.028009288536736458, stdev: 0.45923738706579875, max: 1.4892573356628418, min: -1.3924832344055176)
第 2 個特徵 (count: 1533, mean: 0.003849973568314006, stdev: 0.43567297671586475, max: 1.3220020532608032, min: -1.446252465248108)
第 3 個特徵 (count: 1533, mean: -0.05128098582584903, stdev: 0.4343921399980737, max: 1.4108004570007324, min: -1.6000101566314697)
第 4 個特徵 (count: 1533, mean: -0.11399428229950634, stdev: 0.44183990953013985, max: 1.5220394134521484, min: -1.415898084640503)
第 5 個特徵 (count: 1533, mean: 0.01973237649594426, stdev: 0.4270469439187318, max: 1.2699164152145386, min: -1.339835286140442)


In [ ]:
import numpy as np
def confusion_matrix(GT_Pred):
  GT =GT_Pred.map(lambda x :x[0]).collect()
  Pred=GT_Pred.map(lambda x :x[1]).collect()
  label_list

In [ ]:
from pyspark.mllib.tree import DecisionTree
n_classes=n_classes_
categorical_features_info={}
impurity='gini'
maxdepth=6
maxbins=3
DT_model = DecisionTree.trainClassifier(lp_train,n_classes,categorical_features_info,impurity,maxdepth,maxbins)

#train
GT=lp_train.map(lambda x:x.label)
features=lp_train.map(lambda x:x.features)
Pred = DT_model.predict(features)
GT_Pred = GT.zip(Pred)
c1 =GT_Pred.filter(lambda x:x[0]==x[1]).count()
c2 =GT_Pred.count()
acc=c1/c2
print('Train :acc = ',acc)

#val
GT=lp_val.map(lambda x:x.label)
features=lp_val.map(lambda x:x.features)
Pred = DT_model.predict(features)
GT_Pred = GT.zip(Pred)
c1 =GT_Pred.filter(lambda x:x[0]==x[1]).count()
c2 =GT_Pred.count()
acc=c1/c2
print('Train :acc = ',acc)

#test
GT=lp_test.map(lambda x:x.label)
features=lp_test.map(lambda x:x.features)
Pred = DT_model.predict(features)
GT_Pred = GT.zip(Pred)
c1 =GT_Pred.filter(lambda x:x[0]==x[1]).count()
c2 =GT_Pred.count()
acc=c1/c2
print('test :acc =',acc)

Train :acc =  0.2491846053489889
Train :acc =  0.182648401826484
test :acc = 0.19863013698630136


In [ ]:
from pyspark.mllib.tree import RandomForest
n_classes= n_classes_
categorical_features_info={}
n_trees=100
feature_subset_strategy='auto'
impurity='gini'
maxdepth=8
maxbins=5

RF_model = RandomForest.trainClassifier(lp_train,n_classes,categorical_features_info,n_trees,feature_subset_strategy,impurity,maxdepth,maxbins)

#train
GT=lp_train.map(lambda x:x.label)
features=lp_train.map(lambda x:x.features)
Pred = RF_model.predict(features)
GT_Pred = GT.zip(Pred)
c1 =GT_Pred.filter(lambda x:x[0]==x[1]).count()
c2 =GT_Pred.count()
acc=c1/c2
print('Train :acc = ',acc)

#val
GT=lp_val.map(lambda x:x.label)
features=lp_val.map(lambda x:x.features)
Pred = RF_model.predict(features)
GT_Pred = GT.zip(Pred)
c1 =GT_Pred.filter(lambda x:x[0]==x[1]).count()
c2 =GT_Pred.count()
acc=c1/c2
print('Train :acc = ',acc)

#test
GT=lp_test.map(lambda x:x.label)
features=lp_test.map(lambda x:x.features)
Pred = RF_model.predict(features)
GT_Pred = GT.zip(Pred)
c1 =GT_Pred.filter(lambda x:x[0]==x[1]).count()
c2 =GT_Pred.count()
acc=c1/c2
print('test :acc =',acc)

Train :acc =  1.0
Train :acc =  0.8904109589041096
test :acc = 0.9155251141552512


In [ ]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix
#train
Mat_train = RowMatrix(lp_train.map(lambda x:Vectors.dense(x.features)))
Lab_train = lp_train.map(lambda x:x.label)
pc =Mat_train.computePrincipalComponents(512)
PCA_features =Mat_train.multiply(pc).rows
LP=Lab_train.zip(PCA_features)
LP_train_pca = LP.map(lambda x:LabeledPoint(x[0],x[1]))

#val
Mat_val = RowMatrix(lp_val.map(lambda x:Vectors.dense(x.features)))
Lab_val = lp_val.map(lambda x:x.label)
#pc =Mat_val.computePrincipalComponents(512)
PCA_features =Mat_val.multiply(pc).rows
LP=Lab_val.zip(PCA_features)
LP_val_pca = LP.map(lambda x:LabeledPoint(x[0],x[1]))


#test
Mat_test = RowMatrix(lp_test.map(lambda x:Vectors.dense(x.features)))
Lab_test = lp_test.map(lambda x:x.label)
#pc =Mat_test.computePrincipalComponents(512)
PCA_features =Mat_test.multiply(pc).rows
LP=Lab_test.zip(PCA_features)
LP_test_pca = LP.map(lambda x:LabeledPoint(x[0],x[1]))





In [ ]:
n_classes= n_classes_
categorical_features_info={}
n_trees=100
feature_subset_strategy='auto'
impurity='gini'
maxdepth=6
maxbins=3

RF_model = RandomForest.trainClassifier(LP_train_pca,n_classes,categorical_features_info,n_trees,feature_subset_strategy,impurity,maxdepth,maxbins)

#train
GT=LP_train_pca.map(lambda x:x.label)
features=LP_train_pca.map(lambda x:x.features)
Pred = RF_model.predict(features)
GT_Pred = GT.zip(Pred)
c1 =GT_Pred.filter(lambda x:x[0]==x[1]).count()
c2 =GT_Pred.count()
acc=c1/c2
print('Train :acc = ',acc)

#val
GT=LP_val_pca.map(lambda x:x.label)
features=LP_val_pca.map(lambda x:x.features)
Pred = RF_model.predict(features)
GT_Pred = GT.zip(Pred)
c1 =GT_Pred.filter(lambda x:x[0]==x[1]).count()
c2 =GT_Pred.count()
acc=c1/c2
print('val :acc = ',acc)

#test
GT=LP_test_pca.map(lambda x:x.label)
features=LP_test_pca.map(lambda x:x.features)
Pred = RF_model.predict(features)
GT_Pred = GT.zip(Pred)
c1 =GT_Pred.filter(lambda x:x[0]==x[1]).count()
c2 =GT_Pred.count()
acc=c1/c2
print('test :acc =',acc)


Train :acc =  1.0
val :acc =  0.9223744292237442
test :acc = 0.9018264840182648
